In [16]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
df1 = pd.read_csv("../input/dt-location/DDproperty_2006_data_Extracted - Copy.csv")
df1 = df1.drop(['navlink'], axis = 1)
df1


,Title,Location,Price,bed,bath,listingfloorarea,listingfloorarea2,Type
0,"""ขายด่วน ถูกมาก ปลดภาระ""","ประชาอุทิศ90, บ้านคลองสวน, พระสมุทรเจดีย์, สมุ...",1399000.0,3.0,2.0,110 .,"฿12,718 /",ทาวน์เฮ้าส์
1,"""ขายทาวน์เฮ้าส์ พระราม2 สะแกงาม ทำหน้าร้านขายผ...","4/366 สะแกงาม7, แสมดำ, บางขุนเทียน, กรุงเทพ , ...",1650000.0,2.0,2.0,100 .,"฿16,500 /",ทาวน์เฮ้าส์
2,"""ทาวน์โฮมแฝด 3 ชั้น ม.ลภาวัน17 หลังมุมซอยท่าอิ...","บ้านเลขที่ 200/7 ถนนราชพฤกษ์, บางรักน้อย, เมือ...",5150000.0,4.0,5.0,180 .,"฿28,611 /",ทาวน์เฮ้าส์
3,"""ขายด่วนบ้านเดี่ยว 2 ชั้น เดอะนิช กาญจนทรัพย์ ...","บ้านพรุ, หาดใหญ่, สงขลา , 90110",6600000.0,4.0,4.0,520 .,"฿20,966 /",บ้านเดี่ยว
4,"""มาใหม่ !! ขายคอนโด My Ladprao 27 ถูกที่สุดในโ...","ซอยลาดพร้าว 27 ถนนลาดพร้าว, จันทรเกษม, จตุจักร...",1290000.0,NaN,NaN,25 .,"฿51,600 /",คอนโด
...,...,...,...,...,...,...,...,...
1988,NaN,"281/31 วิภาวดี รังสิต, สนามบิน, ดอนเมือง, กรุง...",4990000.0,3.0,3.0,161 .,"฿30,994 /",ทาวน์เฮ้าส์
1989,NaN,"ุ65/42ซอย14 ลำโพ, ลำโพ, บางบัวทอง, นนทบุรี,11110",2990000.0,3.0,2.0,180 .,"฿13,591 /",บ้านเดี่ยว
1990,NaN,"1 อย.4004, วังพัฒนา, บางซ้าย, พระนครศรีอยุธยา",28855000.0,NaN,NaN,46168 . / 28 ไร่ 3 งาน 42 ตร.วา (ที่ดิน),฿625 /,ที่ดิน
1991,NaN,"- ถนนสุขุมวิท 107, สำโรงเหนือ, เมืองสมุทรปรากา...",5200000.0,2.0,2.0,53 .,"฿97,928 /",คอนโด


In [17]:
#df1

In [18]:
df1 = df1.dropna(how='any',axis=0)
df1

,Title,Location,Price,bed,bath,listingfloorarea,listingfloorarea2,Type
0,"""ขายด่วน ถูกมาก ปลดภาระ""","ประชาอุทิศ90, บ้านคลองสวน, พระสมุทรเจดีย์, สมุ...",1399000.0,3.0,2.0,110 .,"฿12,718 /",ทาวน์เฮ้าส์
1,"""ขายทาวน์เฮ้าส์ พระราม2 สะแกงาม ทำหน้าร้านขายผ...","4/366 สะแกงาม7, แสมดำ, บางขุนเทียน, กรุงเทพ , ...",1650000.0,2.0,2.0,100 .,"฿16,500 /",ทาวน์เฮ้าส์
2,"""ทาวน์โฮมแฝด 3 ชั้น ม.ลภาวัน17 หลังมุมซอยท่าอิ...","บ้านเลขที่ 200/7 ถนนราชพฤกษ์, บางรักน้อย, เมือ...",5150000.0,4.0,5.0,180 .,"฿28,611 /",ทาวน์เฮ้าส์
3,"""ขายด่วนบ้านเดี่ยว 2 ชั้น เดอะนิช กาญจนทรัพย์ ...","บ้านพรุ, หาดใหญ่, สงขลา , 90110",6600000.0,4.0,4.0,520 .,"฿20,966 /",บ้านเดี่ยว
6,"""ถูกสุดในโครงการ Life Ladprao""","992 ถนนลาดพร้าว, จอมพล, จตุจักร, กรุงเทพ, 10900",5300000.0,1.0,1.0,35 .,"฿151,429 /",คอนโด
...,...,...,...,...,...,...,...,...
1661,"""นพอนันต์ เลียบคลองสอง ติดถนนใหญ่ เนื้อที่ 51 ...","ซอย เลียบคลองสอง 16, บางชัน,คลองสามวา, กรุงเทพ...",4390000.0,3.0,3.0,150 .,"฿21,520 /",บ้านเดี่ยว
1662,"""รีโนเวทใหม่ทั้งหลัง สวย ต่อเติมครัว จอดรถ 1 ค...","รังสิต-นครนายก, ลำผักกูด, ธัญบุรี, ปทุมธานี,12110",990000.0,2.0,1.0,96 .,"฿10,312 /",ทาวน์เฮ้าส์
1663,"""บ้านพร้อมพัฒน์ 1 รามอินทรา เนื้อที่ 54.2 ตรว....","ปัญญาเนเชอรัลปาร์ค, สามวาตะวันตก,คลองสามวา, กร...",4900000.0,3.0,3.0,150 .,"฿22,601 /",บ้านเดี่ยว
1664,"""นวลผกา นวลจันทร์ 31 เนื้อที่ 32 ตรว. 4 ชั้น ห...","นวลจันทร์ฺ, นวลจันทร์, บึงกุ่ม, กรุงเทพ,10240",4850000.0,5.0,3.0,290 .,"฿16,724 /",ทาวน์เฮ้าส์



https://www.kaggle.com/code/alexisbcook/nested-and-repeated-data

In [19]:
df1['Location'] = df1['Location'].str.split(",").str[0]
# df['Location'] = df['Location'].str.split(",").str[0]
df1['Location'].str.split(',', 1, expand=True)
df1

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,Title,Location,Price,bed,bath,listingfloorarea,listingfloorarea2,Type
0,"""ขายด่วน ถูกมาก ปลดภาระ""",ประชาอุทิศ90,1399000.0,3.0,2.0,110 .,"฿12,718 /",ทาวน์เฮ้าส์
1,"""ขายทาวน์เฮ้าส์ พระราม2 สะแกงาม ทำหน้าร้านขายผ...",4/366 สะแกงาม7,1650000.0,2.0,2.0,100 .,"฿16,500 /",ทาวน์เฮ้าส์
2,"""ทาวน์โฮมแฝด 3 ชั้น ม.ลภาวัน17 หลังมุมซอยท่าอิ...",บ้านเลขที่ 200/7 ถนนราชพฤกษ์,5150000.0,4.0,5.0,180 .,"฿28,611 /",ทาวน์เฮ้าส์
3,"""ขายด่วนบ้านเดี่ยว 2 ชั้น เดอะนิช กาญจนทรัพย์ ...",บ้านพรุ,6600000.0,4.0,4.0,520 .,"฿20,966 /",บ้านเดี่ยว
6,"""ถูกสุดในโครงการ Life Ladprao""",992 ถนนลาดพร้าว,5300000.0,1.0,1.0,35 .,"฿151,429 /",คอนโด
...,...,...,...,...,...,...,...,...
1661,"""นพอนันต์ เลียบคลองสอง ติดถนนใหญ่ เนื้อที่ 51 ...",ซอย เลียบคลองสอง 16,4390000.0,3.0,3.0,150 .,"฿21,520 /",บ้านเดี่ยว
1662,"""รีโนเวทใหม่ทั้งหลัง สวย ต่อเติมครัว จอดรถ 1 ค...",รังสิต-นครนายก,990000.0,2.0,1.0,96 .,"฿10,312 /",ทาวน์เฮ้าส์
1663,"""บ้านพร้อมพัฒน์ 1 รามอินทรา เนื้อที่ 54.2 ตรว....",ปัญญาเนเชอรัลปาร์ค,4900000.0,3.0,3.0,150 .,"฿22,601 /",บ้านเดี่ยว
1664,"""นวลผกา นวลจันทร์ 31 เนื้อที่ 32 ตรว. 4 ชั้น ห...",นวลจันทร์ฺ,4850000.0,5.0,3.0,290 .,"฿16,724 /",ทาวน์เฮ้าส์


In [20]:
df1 = df1.dropna(how='any',axis=0)
df1

,Title,Location,Price,bed,bath,listingfloorarea,listingfloorarea2,Type
0,"""ขายด่วน ถูกมาก ปลดภาระ""",ประชาอุทิศ90,1399000.0,3.0,2.0,110 .,"฿12,718 /",ทาวน์เฮ้าส์
1,"""ขายทาวน์เฮ้าส์ พระราม2 สะแกงาม ทำหน้าร้านขายผ...",4/366 สะแกงาม7,1650000.0,2.0,2.0,100 .,"฿16,500 /",ทาวน์เฮ้าส์
2,"""ทาวน์โฮมแฝด 3 ชั้น ม.ลภาวัน17 หลังมุมซอยท่าอิ...",บ้านเลขที่ 200/7 ถนนราชพฤกษ์,5150000.0,4.0,5.0,180 .,"฿28,611 /",ทาวน์เฮ้าส์
3,"""ขายด่วนบ้านเดี่ยว 2 ชั้น เดอะนิช กาญจนทรัพย์ ...",บ้านพรุ,6600000.0,4.0,4.0,520 .,"฿20,966 /",บ้านเดี่ยว
6,"""ถูกสุดในโครงการ Life Ladprao""",992 ถนนลาดพร้าว,5300000.0,1.0,1.0,35 .,"฿151,429 /",คอนโด
...,...,...,...,...,...,...,...,...
1661,"""นพอนันต์ เลียบคลองสอง ติดถนนใหญ่ เนื้อที่ 51 ...",ซอย เลียบคลองสอง 16,4390000.0,3.0,3.0,150 .,"฿21,520 /",บ้านเดี่ยว
1662,"""รีโนเวทใหม่ทั้งหลัง สวย ต่อเติมครัว จอดรถ 1 ค...",รังสิต-นครนายก,990000.0,2.0,1.0,96 .,"฿10,312 /",ทาวน์เฮ้าส์
1663,"""บ้านพร้อมพัฒน์ 1 รามอินทรา เนื้อที่ 54.2 ตรว....",ปัญญาเนเชอรัลปาร์ค,4900000.0,3.0,3.0,150 .,"฿22,601 /",บ้านเดี่ยว
1664,"""นวลผกา นวลจันทร์ 31 เนื้อที่ 32 ตรว. 4 ชั้น ห...",นวลจันทร์ฺ,4850000.0,5.0,3.0,290 .,"฿16,724 /",ทาวน์เฮ้าส์


In [21]:
x = df1[['Location','bed','bath','listingfloorarea','listingfloorarea2','Type']]
y = df1['Price']

In [22]:
x['listingfloorarea'] = x['listingfloorarea'].str.replace(' .','')
x['listingfloorarea'] = x['listingfloorarea'].str.replace('3ร.วาที่ดิน','')
x['listingfloorarea'] = x['listingfloorarea'].str.replace(')','')
x['listingfloorarea2'] = x['listingfloorarea2'].str.replace('฿','')
x['listingfloorarea2'] = x['listingfloorarea2'].str.replace('/','')
x['listingfloorarea2'] = x['listingfloorarea2'].str.replace(',','')


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats 

In [23]:
x['listingfloorarea'] = x['listingfloorarea'].astype(str).astype(int)
x['listingfloorarea2'] = x['listingfloorarea2'].astype(str).astype(int)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
from sklearn.preprocessing import LabelEncoder

for c in x.columns:
    if x[c].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(x[c].values))
        x[c] = lbl.transform(x[c].values)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [25]:
x

,Location,bed,bath,listingfloorarea,listingfloorarea2,Type
0,591,3.0,2.0,110,12718,1
1,155,2.0,2.0,100,16500,1
2,584,4.0,5.0,180,28611,1
3,582,4.0,4.0,520,20966,2
6,245,1.0,1.0,35,151429,0
...,...,...,...,...,...,...
1661,315,3.0,3.0,150,21520,2
1662,622,2.0,1.0,96,10312,1
1663,596,3.0,3.0,150,22601,2
1664,521,5.0,3.0,290,16724,1


In [26]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [27]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [28]:
lr = LinearRegression()

knn = KNeighborsRegressor(n_neighbors=10)

dt = DecisionTreeRegressor(max_depth = 3)

rf = RandomForestRegressor(max_depth = 3, n_estimators=500)

ada = AdaBoostRegressor( n_estimators=50, learning_rate =.1)

gbr = GradientBoostingRegressor(max_depth=2, n_estimators=100, learning_rate =.2)

xgb = XGBRegressor(max_depth = 5, n_estimators=50, learning_rate =.1)

cb = CatBoostRegressor(learning_rate =.01, max_depth =5, verbose = 0)

regressors = [('Linear Regression', lr), ('K Nearest Neighbours', knn),
               ('Decision Tree', dt), ('Random Forest', rf), ('AdaBoost', ada),
              ('Gradient Boosting Regressor', gbr), ('XGBoost', xgb), ('catboost', cb)]

In [29]:
from sklearn.metrics import r2_score

for regressor_name, regressor in regressors:
 
    # Fit regressor to the training set
    regressor.fit(x_train, y_train)    
   
    # Predict 
    y_pred = regressor.predict(x_test)
    accuracy = round(r2_score(y_test,y_pred),2)*100
    
    

   
    # Evaluate  accuracy on the test set
    print('{:s} : {:.0f} %'.format(regressor_name, accuracy))

Linear Regression : 16 %
K Nearest Neighbours : 19 %
Decision Tree : 5 %
Random Forest : 51 %
AdaBoost : 30 %
Gradient Boosting Regressor : 71 %
XGBoost : 67 %
catboost : 60 %
